In [31]:
import pandas as pd
import json
import time
import random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pathlib import Path

In [32]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

In [33]:
source_urls = "sources.csv"
output_file = "basket.csv"

urls = []
with open(source_urls, 'r') as file:
    urls = [url.strip() for url in file if url.strip()]
    urls = [url.split('?')[0] for url in urls]
    seen = set()
    urls = [url for url in urls if not (url in seen or seen.add(url))]

df_products = []
current_date = datetime.now().strftime("%Y-%m-%d")

for i, url in enumerate(urls, 1):
    try:
        print(f"[{i}/{len(urls)}] Processing: {url}")
        driver.get(url)
        
        # Random sleep between 0.5 and 3.0 seconds, rounded to 1 decimal place
        sleep_time = round(random.uniform(0.5, 3.0), 1)
        time.sleep(sleep_time)

        jace = driver.find_element(By.ID, "__NEXT_DATA__").get_attribute("innerHTML")
        jace_data = json.loads(jace)['props']['pageProps']['productDetails']['children'][0]

        product = {
            "Record Date": current_date,
            "Category": jace_data['category']['tlc_name'],
            "Brand": jace_data['brand']['name'],
            "Product": jace_data['desc'],
            "Quantity": jace_data['w'],
            "Price-MRP": float(jace_data['pricing']['discount']['mrp']),
            "Price-SP": float(jace_data['pricing']['discount']['prim_price']['sp'])
        }

        df_products.append(product)

    except Exception as e:
        print(f"Error processing URL {url}: {str(e)}\n")

driver.quit()

print("\nToday's data summary:")
df_products = pd.DataFrame(df_products)
display(df_products)

[1/14] Processing: https://www.bigbasket.com/pd/40001002/surf-excel-quick-wash-detergent-powder-2-kg/
[2/14] Processing: https://www.bigbasket.com/pd/271205/id-fresho-idly-dosa-batter-1-kg/
[3/14] Processing: https://www.bigbasket.com/pd/40189244/kelloggs-muesli-breakfast-cereal-with-multigrain-21-fruit-nut-seeds-750-g-0/
[4/14] Processing: https://www.bigbasket.com/pd/30003294/id-fresho-malabar-parotaparatha-350-g-pouch/
[5/14] Processing: https://www.bigbasket.com/pd/40092421/kelloggs-corn-flakes-with-real-almond-honey-1-kg/
[6/14] Processing: https://www.bigbasket.com/pd/40184646/bb-royal-organic-himalayan-kashmir-honey-500-g/
[7/14] Processing: https://www.bigbasket.com/pd/40282752/koparo-clean-laundry-liquid-lavender-lilac-bergamot-non-toxic-gentle-on-fabrics-18-l/
[8/14] Processing: https://www.bigbasket.com/pd/40307614/prestige-svachh-flip-on-hard-anodised-pressure-cooker-with-glass-lid-gas-induction-compatible-black-5-l/
[9/14] Processing: https://www.bigbasket.com/pd/40199889/

,Record Date,Category,Brand,Product,Quantity,Price-MRP,Price-SP
0,2025-07-25,Cleaning & Household,Surf Excel,Quick Wash Detergent Powder,2 kg,450.0,450.00
1,2025-07-25,Snacks & Branded Foods,iD fresho!,Idly & Dosa Batter,1 kg,100.0,99.00
2,2025-07-25,Snacks & Branded Foods,Kellogg's,Muesli Fruit Nut & Seeds - 12 In 1 Power Break...,750 g,525.0,448.88
3,2025-07-25,Snacks & Branded Foods,iD fresho!,Malabar Parota - No Added Preservatives,400 g,110.0,108.90
4,2025-07-25,Snacks & Branded Foods,Kellogg's,Corn Flakes - With Real Almond & Honey,1 kg,580.0,477.98
5,2025-07-25,Snacks & Branded Foods,bb Royal,Organic Kashmir Honey,500 g,429.0,307.00
6,2025-07-25,Cleaning & Household,Koparo Clean,"Liquid Detergent - Lavender, Lilac & Bergamot ...",2 L,585.0,585.00
7,2025-07-25,"Kitchen, Garden & Pets",Prestige,Svachh Flip-On Hard-Anodised Gas & Induction C...,5 L,5290.0,2579.00
8,2025-07-25,"Kitchen, Garden & Pets",Asian,"Storage Container - Assorted Colour, Plastic, ...",800 ml,201.0,169.00
9,2025-07-25,Beauty & Hygiene,Gillette,Fusion Power Blades for men Perfect Shave and...,8 pcs,2399.0,2231.00


In [34]:
file_path = Path(output_file)
if file_path.exists():
    try:
        existing_df = pd.read_csv(output_file)
        combined_df = pd.concat([existing_df, df_products], ignore_index=True)
        combined_df = combined_df.sort_values('Record Date').drop_duplicates(
            subset=['Record Date', 'Brand', 'Product'], keep='last')
        combined_df.to_csv(output_file, index=False)

    except Exception as e:
        print(f"Error processing existing file: {str(e)}")
        print("Creating new file with current data only.")
        df_products.to_csv(output_file, index=False)

else:
    df_products.to_csv(output_file, index=False)
    print(f"Created new file with {len(df_products)} records.")

# display(pd.read_csv(output_file))

In [35]:
df = pd.read_csv(output_file)
df = df.sort_values(by=['Category', 'Brand', 'Product', 'Record Date'], ascending=True).reset_index(drop=True)
display(df)

,Record Date,Category,Brand,Product,Quantity,Price-MRP,Price-SP
0,2025-03-24,Beauty & Hygiene,Gillette,Fusion Power Blades for men Perfect Shave and...,8 pcs,2399.0,2399.00
1,2025-07-25,Beauty & Hygiene,Gillette,Fusion Power Blades for men Perfect Shave and...,8 pcs,2399.0,2231.00
2,2025-03-24,Beauty & Hygiene,Nivea,Nourishing Body Milk,400 ml,399.0,279.30
3,2025-07-25,Beauty & Hygiene,Nivea,Nourishing Body Milk,400 ml,425.0,347.00
4,2025-03-24,Cleaning & Household,Koparo Clean,"Liquid Detergent - Lavender, Lilac & Bergamot ...",2 L,585.0,427.05
5,2025-07-25,Cleaning & Household,Koparo Clean,"Liquid Detergent - Lavender, Lilac & Bergamot ...",2 L,585.0,585.00
6,2025-03-24,Cleaning & Household,Surf Excel,Quick Wash Detergent Powder,2 kg,450.0,423.00
7,2025-07-25,Cleaning & Household,Surf Excel,Quick Wash Detergent Powder,2 kg,450.0,450.00
8,2025-03-24,"Eggs, Meat & Fish",Farm Made,Free Range Eggs,24 pcs,569.0,569.00
9,2025-07-25,"Eggs, Meat & Fish",Farm Made,Free Range Eggs,24 pcs,569.0,540.55


In [36]:
df.groupby('Record Date').agg({
    'Price-MRP': 'sum',
    'Price-SP': 'sum'
})

,Price-MRP,Price-SP
Record Date,,
2025-03-24,12259.0,10552.76
2025-07-25,12295.0,8902.31
